In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sat Dec  9 00:01:55 2023

@author: Aditya Pavadad
"""

"""
"""

# ---- Import libraries ----
import matplotlib.pyplot as plt
from gurobipy import *
import numpy as np
import csv


# ---- Parameters ----

"""Importing the dataset and extracting the appropriate data"""

filename = "data_large_3dep.txt"


with open(filename) as f:
    data = csv.reader(f, delimiter="\t")
    dataset = list(data)

for i in range(0,len(dataset)):
    for j in range(0,7):
        a = int(dataset[i][j])
        dataset[i][j] = a
        
dataset = np.array(dataset)
depot = dataset[:3]
customer = dataset[3:]
node_id = dataset[:, 0]  # Stores the id's of the nodes
x_coord = dataset[:, 1]  # Stores the x coordinates of all the nodes
y_coord = dataset[:, 2]  # Stores the y coordinates of all the nodes
a = dataset[:, 3]  # Stores the demand for each node
RT = dataset[:, 4]  # Stores the ready time for each node (earliest time for delivery)
DT = dataset[:, 5]  # Stores the due time for each node (latest time for delivery)
ST = dataset[:, 6]  # Stores the service time for each node (time needed for a delivery at each location )



"""Parameters"""


# Capacities for each vehicle, with the commented out sections being for each case

b = np.array([300 , 300 , 300 , 300 , 300 , 300])  # Case 1
Dc = np.array([5, 0, 0])


# Total number of vehicles
K = len(b)

"""Calculating the Euclidean distance between all nodes """

D = np.zeros((len(a), len(a)))


for i in range(len(node_id)):
    for j in range(len(node_id)):
        D[i, j] = np.linalg.norm((x_coord[i] - x_coord[j], y_coord[i] - y_coord[j]))
        

# Note for this programme, the assumption is made that 1 distance unit corresponds to 1 time unit
# This also allows us to use distance interchangably within constraints with time


# ---- Initialising Model ----

model = Model("Part c/d")


# ---- Sets and Indices ----

I = range(len(node_id))  # locations
J = range(len(node_id))  # locations
V = range(K)  # vehicles

# ---- Decision Variables ----

"""Initialising the dimensions for all the decision variables"""
x = model.addVars(I, J, V, vtype = GRB.BINARY, name = "If_Route_Travelled")
z = model.addVars(I, V, vtype = GRB.BINARY, name = "If_Node_Satisfied")
t = model.addVars(I, V, vtype = GRB.CONTINUOUS, name = "Service_Start_Time")
N = model.addVars(range(3), lb = 0, ub = 3, vtype = GRB.INTEGER, name = "Vehicle_Per_Depot")
    
    
# # ---- Integerate the variables into the model ----
model.update()


# # ---- Objective Function ----
obj = quicksum(x[i, j, v]*D[i, j] for i in I for j in J for v in V)
model.setObjective(obj, GRB.MINIMIZE)  # Only a function of distance as that is the parameter to minimize

model.update()

# # ---- Constraints ----

M = 4000

# # Ensures that every node is visited by only one vehicle
single_visit = model.addConstrs((quicksum(z[i, v] for v in V) == 1 for i in range(3, len(a))), 
                                "Single_Visit_Satisfaction")

# # Ensures that multiple vehicles can start from the Depot
depot_vehicle_num = model.addConstrs((quicksum(z[i, v] for v in V) <= N[i] for i in range(3)), 
                                     "Vehicle_Leaving_Depot")
    
# # Total Number of Vehicles leaving the depots is K
total_vehicles = model.addConstr(quicksum(N[i] for i in range(3)) == K, "Total_Vehicles")    

# # Ensures that the sum of demand of a tour of each vehicle needs to be less than overall capacity of the vehicle
vehicle_cap = model.addConstrs((quicksum(a[i]*z[i, v] for i in I) <= b[v] for v in V), "Vehicle_Capacity")

# # Ensures that the number of vehicles leaving a node equals number of vehicles satisfying its demands
vehicle_departure = model.addConstrs((quicksum(x[i, j, v] for j in J) == z[i, v] for i in I for v in V), 
                                     "Vehicle_Departure_Constraint")

# # Ensures that each nodes is visited by particular vehicle v once
vehicle_arrival = model.addConstrs((quicksum(x[i, j, v] for i in I) == z[j, v] for j in J for v in V), 
                                   "Vehicle_Arrival_Constraint")

# # Defining the total time taken up to a certain node for a certain vehicle
# service_start = model.addConstrs((quicksum(x[i, j, v]*(t[i, v] + ST[i] + D[i, j]) for i in I) <= t[j, v] 
#                                   for j in range(3, len(a)) for v in V), 
#                                  "Service_Start_Time_Constraint")

# Defining the total time taken up to a certain node for a certain vehicle with Big M
# service_start = model.addConstrs((t[j, v] >= quicksum((t[i, v] + ST[i] + D[i, j]) * x[i, j, v] for i in I)
#                                   - M * (1 - x[i, j, v]) for j in range(3, len(a)) for v in V),
#                                  "Service_Start_Time_Constraint_Big_M")



# # Ensures that there is no subtouring
# subtour_constraint = model.addConstrs((t[i, v] + D[i, j] - M*(1 - x[i, j, v]) <= t[j, v] 
#                                        for i in I for j in range(3, len(a)) for v in V), "Subtour_Constraint")

# subtour_constraint = model.addConstrs((t[i, v] + D[i, j] + ST[i] - M*(1 - x[i, j, v]) <= t[j, v] 
#                                        for i in I for j in range(3, len(a)) for v in V), "Subtour_Constraint")

# m.addConstr(u[0] == 0)
# m.addConstr(u[1] == 0)
# m.addConstr(u[2] == 0)
# for i in I:
#     for i not in [0 , 1 , 2]:
#         m.addConstr(u[i] >= a[i])
#         m.addConstr(u[i] <= b )

# for v in V:
#     for i in I:
#         for j in J:
#             if j not in [0 , 1 , 2] and i not in [0,1,2] and i! = j:
#                 m.addConstr(u[j] - u[i] + M * (1 - x[ i , j , v ] > a[j]))
                
# for i in I:
#     for v in V:
#         m.addConstr(x[i , i , j ] , GRB.EQUAL , 0 , name = "TEST_NO_RETURN")



subtour_constraint = model.addConstrs((t[i, v] + D[i, j] + ST[i] - M * (1 - x[i, j, v]) <= t[j, v]
                                       for i in I for j in range(3, len(a)) for v in V), "Subtour_Constraint")

self_return = model.addConstrs((x[i, i, v] == 0 for i in I for v in V), "No_Self_Return_Constraint")


# # Ensures that the arrival of the vehicle is within the ready-time and due-time
ready_time_constraint = model.addConstrs((RT[i]*z[i, v] <= t[i, v] for i in I for v in V), "Ready_Time_Constraint")
due_time_constraint = model.addConstrs((t[i, v]*z[i, v] <= DT[i] for i in I for v in V), "Due_Time_Constraint")

# # ---- Gurobi Parameters ----
# """Ensuring that the optimal solution is found and not a heuristic"""

model.update()

# model.setParam("OutputFlag", False)  # silencing gurobi output or not
# model.setParam ('Heuristic', 0.5)
model.setParam('MIPGap', 2)
model.setParam('TimeLimit', 1800)

# find the optimal solution
model.write("output.lp")  # print the model in .lp format file

model.optimize()




Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-14
Set parameter MIPGap to value 2
Set parameter TimeLimit to value 1800
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 17232 rows, 17493 columns and 83004 nonzeros
Model fingerprint: 0x4258cc85
Model has 318 quadratic constraints
Variable types: 318 continuous, 17175 integer (17172 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 3e+00]
  RHS range        [1e+00, 4e+03]
  QRHS range       [4e+02, 1e+03]
Presolve removed 636 rows and 318 columns
Presolve time: 0.11s
Presolved: 17550 rows, 18129 columns, 83640 nonzeros
Presolved model has 636 SOS constraint(s)
Variable types: 954 continuous, 17175 int

     0     0  171.62114    0  557          -  171.62114      -     -   42s
     0     0  171.62293    0  588          -  171.62293      -     -   42s
     0     0  171.62494    0  555          -  171.62494      -     -   42s
     0     0  171.63249    0  519          -  171.63249      -     -   42s
     0     0  171.74422    0  575          -  171.74422      -     -   43s
     0     0  171.74422    0  601          -  171.74422      -     -   43s
     0     0  171.74478    0  576          -  171.74478      -     -   43s
     0     0  171.79026    0  611          -  171.79026      -     -   43s
     0     0  171.93759    0  596          -  171.93759      -     -   43s
     0     0  171.93851    0  597          -  171.93851      -     -   44s
     0     0  171.93851    0  615          -  171.93851      -     -   45s
     0     2  173.09763    0  612          -  173.09763      -     -   50s
   187   200  188.05905   19  393          -  186.57078      -  54.8   55s
   417   445  190.03721  